<a href="https://colab.research.google.com/github/rushithakondreddy/haihua_INFO5731_Spring2020/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [11]:
# Write your code here
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from xgboost import XGBClassifier

In [12]:
train_df = pd.read_csv(r'/content/stsa-train.txt',sep = 'delimiter=',header= None,names=['Text'])
train_df[['Sentiment','Text']] = train_df["Text"].str.split(" ", 1, expand=True)
test_df = pd.read_csv(r'/content/stsa-test.txt',sep = 'delimiter=',header= None,names=['Text'])
test_df[['Sentiment','Text']] = test_df["Text"].str.split(" ", 1, expand=True)
train_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,Text,Sentiment
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [13]:
test_df.head()

,Text,Sentiment
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [14]:
# Clean data
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()
def clean_text(data):
  data="".join([word.lower() for word in data if word not in string.punctuation])
  data = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", data)
  tokens = re.split('\W+',data)
  data = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
# Convert text and train data into numerical
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train_df['Text'])
print(X_tfidf.shape)
X_tfidf_df=pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns=tfidf_vect.get_feature_names()
X_test_tfidf = tfidf_vect.transform(test_df['Text'])
print(X_test_tfidf.shape)

(6920, 13343)
(1821, 13343)


In [16]:
#(1) MultinominalNB
mnb = MultinomialNB()
#(2) SVM
svm = LinearSVC()
#(3) KNN
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
#(4) Decision tree
dt = DecisionTreeClassifier()
#(5) Random Forest
rf = RandomForestClassifier()
#(6) XGBoost
xgb = XGBClassifier()

#split the data
x_train, x_test, y_train, y_test = train_test_split(X_tfidf_df, train_df['Sentiment'].values,
                                                test_size=0.2, random_state=42)

In [17]:
#(1) MultinominalNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
mnb_test = mnb.fit(x_train,y_train)
y_mnb = mnb_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_mnb,y_test))
print(classification_report(y_test,y_mnb))
from sklearn.model_selection import cross_val_score
scores_MNB = cross_val_score(mnb, x_test, y_test, cv=10)
print("Accuracy with MNB",scores_MNB.mean())

Accuracy Score 0.7955202312138728
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       671
           1       0.76      0.88      0.82       713

    accuracy                           0.80      1384
   macro avg       0.80      0.79      0.79      1384
weighted avg       0.80      0.80      0.79      1384

Accuracy with MNB 0.7247054530288813


In [18]:
#(2) SVM
svm_test = svm.fit(x_train,y_train)
y_svm = svm_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_svm,y_test))
print(classification_report(y_test,y_svm))
from sklearn.model_selection import cross_val_score
scores_SVM = cross_val_score(svm, x_test, y_test, cv=10)
print("Accuracy with SVM",scores_SVM.mean())

Accuracy Score 0.791907514450867
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       671
           1       0.78      0.83      0.80       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384

Accuracy with SVM 0.7348034615785632


In [19]:
#(3) KNN
knn_test = knn.fit(x_train,y_train)
y_knn = knn_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_knn,y_test))
print(classification_report(y_test,y_knn))
from sklearn.model_selection import cross_val_score
scores_KNN = cross_val_score(knn, x_test, y_test, cv=10)
print("Accuracy with knn",scores_KNN.mean())

Accuracy Score 0.7398843930635838
              precision    recall  f1-score   support

           0       0.74      0.70      0.72       671
           1       0.74      0.77      0.75       713

    accuracy                           0.74      1384
   macro avg       0.74      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384

Accuracy with knn 0.6675737670732979


In [20]:
#(4) Decision tree
des_tree_test = dt.fit(x_train,y_train)
y_des_tree = des_tree_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_des_tree,y_test))
print(classification_report(y_test,y_des_tree))
scores_DT = cross_val_score(dt, x_test, y_test, cv=10)
print("Accuracy with Decision trees",scores_DT.mean())


Accuracy Score 0.6690751445086706
              precision    recall  f1-score   support

           0       0.67      0.63      0.65       671
           1       0.67      0.71      0.69       713

    accuracy                           0.67      1384
   macro avg       0.67      0.67      0.67      1384
weighted avg       0.67      0.67      0.67      1384

Accuracy with Decision trees 0.6090605776248565


In [21]:
#(5) Random Forest
rf_test = rf.fit(x_train,y_train)
y_rf = rf_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_rf,y_test))
print(classification_report(y_test,y_rf))
scores_RF = cross_val_score(rf, x_test, y_test, cv=10)
print("Accuracy with Random Forest",scores_RF.mean())

Accuracy Score 0.7384393063583815
              precision    recall  f1-score   support

           0       0.79      0.63      0.70       671
           1       0.71      0.84      0.77       713

    accuracy                           0.74      1384
   macro avg       0.75      0.74      0.73      1384
weighted avg       0.75      0.74      0.74      1384

Accuracy with Random Forest 0.6885569805025545


In [22]:
#(6) XGBoost
xgb_test = xgb.fit(x_train,y_train)
y_xgb = xgb_test.predict(x_test)
print('Accuracy Score %s' % accuracy_score(y_xgb,y_test))
print(classification_report(y_test,y_xgb))
scores_XGB = cross_val_score(xgb, x_test, y_test, cv=10)
print("Accuracy with XGBoost",scores_XGB.mean())

Accuracy Score 0.6445086705202312
              precision    recall  f1-score   support

           0       0.75      0.40      0.52       671
           1       0.61      0.88      0.72       713

    accuracy                           0.64      1384
   macro avg       0.68      0.64      0.62      1384
weighted avg       0.68      0.64      0.62      1384

Accuracy with XGBoost 0.6184704410384736


In [23]:
print("Accuracy with MNB",scores_MNB.mean())
print("Accuracy with SVM",scores_SVM.mean())
print("Accuracy with knn",scores_KNN.mean())
print("Accuracy with Decision trees",scores_DT.mean())
print("Accuracy with Random Forest",scores_RF.mean())
print("Accuracy with XGBoost",scores_XGB.mean())

Accuracy with MNB 0.7247054530288813
Accuracy with SVM 0.7348034615785632
Accuracy with knn 0.6675737670732979
Accuracy with Decision trees 0.6090605776248565
Accuracy with Random Forest 0.6885569805025545
Accuracy with XGBoost 0.6184704410384736


In [24]:
predict_mnb = mnb_test.predict(X_test_tfidf)
print('Final trained model(MNB) with high accuracy evaluated on the test data: %s' % accuracy_score(predict_mnb,test_df['Sentiment']))

Final trained model(MNB) with high accuracy evaluated on the test data: 0.7946183415705657
